In [1]:
'''
as part of MS4 green infrastructure equity work and following this guidance from MA: 
https://www.mass.gov/doc/2016-massachusetts-small-ms4-permit-pollutant-loading-export-rates/download

generating PLER values for each land cover area in each municipality

created: 5/2/2023
author: rbowers

'''

import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import sys
sys.path.append("..")
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import contextily as cx
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import geopandas as gpd

import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import rioxarray as rxr
import xarray as xr 
from numpy import int16
from shapely.geometry import mapping
import pysheds
from pysheds.grid import Grid
from shapely.validation import make_valid



#see all columns in tables
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
from src.features.build_features import *

In [2]:
munis_list = ['Franklin', 'Norwood', 'Reading', 'Winchester', 'Natick']


for muni in munis_list:

    #read in municipalities 
    munis_fp = "K:\\\DataServices\\Datasets\\Boundaries\\Spatial\\mapc_towns_poly.shp"
    ms4_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb'
    ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'

    munis = gpd.read_file(munis_fp)

    #select just the muni 
    muni_shp = munis.loc[munis['municipal'] == muni]

    print('Reading in Land Cover/Land Use Data for ' + muni + '...') 

    #land use geodatabase location
    lclu = gpd.read_file(ms4_model_gdb, layer="lclu_simplify_all_mapc", mask=muni_shp)
    
    print('Validating land use geometry...')
    lclu.geometry = lclu.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

    print('Reading in Soils Data for ' + muni + '...')

    #soils geodabatse location
    
    soils_hsg = gpd.read_file(ms4_gdb, layer="soils_mapc_simplify")

    print('Validating soils geometry...')
    soils_hsg.geometry = soils_hsg.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

    print('Reprojecting all to Mass Mainland')
    #reproject all to mass mainland
    mass_mainland_crs = "EPSG:26986"
    lclu = lclu.to_crs(mass_mainland_crs)
    muni_shp = muni_shp.to_crs(mass_mainland_crs)
    soils_hsg = soils_hsg.to_crs(mass_mainland_crs)

    #start with anything that is water and not water
    lclu_water = lclu.loc[(lclu['covercode'] == 21) | (lclu['covercode'] == 22)] #21 is cover code for water, 22 for aquatic bed
    lclu_not_water = lclu.loc[(lclu['covercode'] != 21) & (lclu['covercode'] != 22)]

    #then anything that is impervious and not impervious
    lclu_impervious = lclu_not_water.loc[lclu_not_water['covercode'] == 2] #2 is cover code for imperviousness
    lclu_pervious = lclu_not_water.loc[lclu_not_water['covercode'] != 2]

    #get pler value for impervious surfaces

    imp_pler_rule = [
        (lclu_impervious['usegencode'] == 10) | (lclu_impervious['usegencode'] == 12),
        (lclu_impervious['usegencode'] == 3) | (lclu_impervious['usegencode']==4) | (lclu_impervious['usegencode']==9) | (lclu_impervious['usegencode']==20) | (lclu_impervious['usegencode']==30),
        lclu_impervious['usegencode']==55,
        (lclu_impervious['usegencode'] == 6) | (lclu_impervious['usegencode']==7) | (lclu_impervious['usegencode']==0) | (lclu_impervious['usegencode']==2) | (lclu_impervious['usegencode']==8),
        (lclu_impervious['usegencode'] == 13) | (lclu_impervious['usegencode']==11)
        ]

    choices = [2.32, 1.78, 1.95, 1.52, 1.96]

    lclu_impervious['pler'] = np.select(imp_pler_rule, choices, default=np.nan).astype(np.float32)

    #get pler value for pervious, leaving a value of 0 for the land cover classes that require hsg
    pervious_pler_rule = [
        (lclu_pervious['covercode'] == 9) | (lclu_pervious['covercode'] ==10) | (lclu_pervious['covercode']==12),
        (lclu_pervious['usegencode'] == 6) & (
                        (lclu_pervious['covercode']==5) | (lclu_pervious['covercode']==8) | 
                        (lclu_pervious['covercode']==13) | (lclu_pervious['covercode']==14)| 
                        (lclu_pervious['covercode']==15) | (lclu_pervious['covercode']==16) |
                        (lclu_pervious['covercode']==17)|(lclu_pervious['covercode']==19) |(lclu_pervious['covercode']==20)),
        (lclu_pervious['covercode'] == 6) | (lclu_pervious['covercode']==7),
        (lclu_pervious['covercode'] == 5) & (lclu_pervious['usegencode']== 7),
        (lclu_pervious['covercode'] == 8) & (lclu_pervious['usegencode']== 7),
        (lclu_pervious['covercode'] == 7) & (lclu_pervious['usegencode']== 6),
        (lclu_pervious['covercode']==5) | (lclu_pervious['covercode']==8) | (lclu_pervious['covercode']==13) 
            | (lclu_pervious['covercode']==14) | (lclu_pervious['covercode']==15) | (lclu_pervious['covercode']==16) | 
            (lclu_pervious['covercode']==17) | (lclu_pervious['covercode']==18) | 
            (lclu_pervious['covercode']==19) | (lclu_pervious['covercode']==20)
        ]

    choices = [0.13, 0.13, 0.45, 0.45, 0.45, 0.45, 0.00]

    lclu_pervious['pler'] = np.select(pervious_pler_rule, choices, default=np.nan).astype(np.float32)

    #now separate into what HAS a pler or what now needs to be calculated based on soils
    lclu_pervious_withpler = lclu_pervious.loc[lclu_pervious['pler'] > 0]
    lclu_pervious_nopler = lclu_pervious.loc[lclu_pervious['pler'] == 0]

    #now break down the no_pler pervious gdf into what HAS a hydrologic soil group and what does not

    #clip soils polygon to the pervious_no pler gdf
    soils_clip = soils_hsg[['HYDROLGRP', 'COMPNAME', 'geometry']].clip(lclu_pervious_nopler, keep_geom_type=True)


    #we can get pler from the soil layer if it has a HSG or if it's Urban or Udorthent soil
    soil_types = ['Udorthents', 'Urban land']
    soils_clip_withhsg_or_urbanland = soils_clip.loc[(soils_clip['HYDROLGRP'] != ' ') | ((soils_clip['HYDROLGRP'] == ' ') & (soils_clip['COMPNAME'].isin(soil_types)))]

    #if the soil doesn't have a hsg and it's not urban or udorthent soil, AND it's rock outcrop, we get the pler by land use
    soils_clip_rockoutcrop = soils_clip.loc[((soils_clip['HYDROLGRP'] == ' ') & (soils_clip['COMPNAME'] == 'Rock outcrop'))]


    #first get pler for hsg / urban land
    hsg_pler_rule = [
            (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'A') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'A/D'),
            (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'B') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'B/D'),
            (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'C') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'C/D'),
            soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'D',
            soils_clip_withhsg_or_urbanland['HYDROLGRP'] == ' ' #this captures urban land or udorthent soils
    ]

    choices = [0.03, 0.12, 0.21, 0.37, 0.21]

    soils_clip_withhsg_or_urbanland['pler'] = np.select(hsg_pler_rule, choices, default=np.nan).astype(np.float32)


    #to get pler for rock outcrop areas, clip lclu and run for that 
    lclu_pervious_rockoutcrop = lclu_pervious_nopler.clip(soils_clip_rockoutcrop, keep_geom_type=True)

    imp_pler_rule = [
        (lclu_pervious_rockoutcrop['usegencode'] == 10) | (lclu_pervious_rockoutcrop['usegencode'] == 12),
        (lclu_pervious_rockoutcrop['usegencode'] == 3) | (lclu_pervious_rockoutcrop['usegencode']==4) | (lclu_pervious_rockoutcrop['usegencode']==9) | 
            (lclu_pervious_rockoutcrop['usegencode']==20) | (lclu_pervious_rockoutcrop['usegencode']==30),
        lclu_pervious_rockoutcrop['usegencode']==55,
        (lclu_pervious_rockoutcrop['usegencode'] == 6) | (lclu_pervious_rockoutcrop['usegencode']==7) | (lclu_pervious_rockoutcrop['usegencode']==0) | 
            (lclu_pervious_rockoutcrop['usegencode']==2) | (lclu_pervious_rockoutcrop['usegencode']==8),
        (lclu_pervious_rockoutcrop['usegencode'] == 13) | (lclu_pervious_rockoutcrop['usegencode']==11)
        ]

    choices = [2.32, 1.78, 1.95, 1.52, 1.96]

    lclu_pervious_rockoutcrop['pler'] = np.select(imp_pler_rule, choices, default=np.nan).astype(np.float32)

    #join and dissolve
    joined = pd.concat([lclu_pervious_rockoutcrop[['pler', 'geometry']], 
                        soils_clip_withhsg_or_urbanland[['pler', 'geometry']], 
                        lclu_pervious_withpler[['pler', 'geometry']], 
                        lclu_impervious[['pler', 'geometry']]])

    print('Dissolving pler data for ' + muni + '....')
    pler_merge_dissolve = joined.dissolve(by='pler').reset_index()

    print('Exporting shapefiles for  ' + muni + '....')
    folder_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Intermediate"
    path = os.path.join(folder_path, muni) #make a subdirectory in intermediate folder w town name
    os.makedirs(path, exist_ok=True)
    pler_merge_dissolve.to_file(path + '\\pler_dissolve.shp')

    lclu_impervious_dissolve = lclu_impervious.dissolve(by='pler').reset_index()
    lclu_impervious_dissolve.to_file(path + '\\pler_imp_dissolve.shp')


Reading in Land Cover/Land Use Data for Franklin...
Validating land use geometry...
Reading in Soils Data for Franklin...


KeyboardInterrupt: 

In [2]:

#read in municipalities 
subregions_fp = 'K:\\DataServices\\Datasets\\Boundaries\\Spatial\\mapc_subregions.shp'
ms4_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb'
ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'


print('Reading in Land Cover/Land Use Data...') 

#land use geodatabase location
lclu = gpd.read_file(ms4_model_gdb, layer="lclu_simplify_all_mapc_1")

#print('Validating land use geometry...')
#lclu.geometry = lclu.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

print('Reading in Soils Data for...')

#soils geodabatse location

soils_hsg = gpd.read_file(ms4_gdb, layer="soils_mapc_simplify")


print('Reprojecting all to Mass Mainland')
#reproject all to mass mainland
mass_mainland_crs = "EPSG:26986"
lclu = lclu.to_crs(mass_mainland_crs)
soils_hsg = soils_hsg.to_crs(mass_mainland_crs)

#start with anything that is water and not water
lclu_water = lclu.loc[(lclu['covercode'] == 21) | (lclu['covercode'] == 22)] #21 is cover code for water, 22 for aquatic bed
lclu_not_water = lclu.loc[(lclu['covercode'] != 21) & (lclu['covercode'] != 22)]

#then anything that is impervious and not impervious
lclu_impervious = lclu_not_water.loc[lclu_not_water['covercode'] == 2] #2 is cover code for imperviousness
lclu_pervious = lclu_not_water.loc[lclu_not_water['covercode'] != 2]

#get pler value for impervious surfaces

imp_pler_rule = [
    (lclu_impervious['usegencode'] == 10) | (lclu_impervious['usegencode'] == 12),
    (lclu_impervious['usegencode'] == 3) | (lclu_impervious['usegencode']==4) | (lclu_impervious['usegencode']==9) | (lclu_impervious['usegencode']==20) | (lclu_impervious['usegencode']==30),
    lclu_impervious['usegencode']==55,
    (lclu_impervious['usegencode'] == 6) | (lclu_impervious['usegencode']==7) | (lclu_impervious['usegencode']==0) | (lclu_impervious['usegencode']==2) | (lclu_impervious['usegencode']==8),
    (lclu_impervious['usegencode'] == 13) | (lclu_impervious['usegencode']==11)
    ]

choices = [2.32, 1.78, 1.95, 1.52, 1.96]

lclu_impervious['pler'] = np.select(imp_pler_rule, choices, default=np.nan).astype(np.float32)

#get pler value for pervious, leaving a value of 0 for the land cover classes that require hsg
pervious_pler_rule = [
    (lclu_pervious['covercode'] == 9) | (lclu_pervious['covercode'] ==10) | (lclu_pervious['covercode']==12),
    (lclu_pervious['usegencode'] == 6) & (
                    (lclu_pervious['covercode']==5) | (lclu_pervious['covercode']==8) | 
                    (lclu_pervious['covercode']==13) | (lclu_pervious['covercode']==14)| 
                    (lclu_pervious['covercode']==15) | (lclu_pervious['covercode']==16) |
                    (lclu_pervious['covercode']==17)|(lclu_pervious['covercode']==19) |(lclu_pervious['covercode']==20)),
    (lclu_pervious['covercode'] == 6) | (lclu_pervious['covercode']==7),
    (lclu_pervious['covercode'] == 5) & (lclu_pervious['usegencode']== 7),
    (lclu_pervious['covercode'] == 8) & (lclu_pervious['usegencode']== 7),
    (lclu_pervious['covercode'] == 7) & (lclu_pervious['usegencode']== 6),
    (lclu_pervious['covercode']==5) | (lclu_pervious['covercode']==8) | (lclu_pervious['covercode']==13) 
        | (lclu_pervious['covercode']==14) | (lclu_pervious['covercode']==15) | (lclu_pervious['covercode']==16) | 
        (lclu_pervious['covercode']==17) | (lclu_pervious['covercode']==18) | 
        (lclu_pervious['covercode']==19) | (lclu_pervious['covercode']==20)
    ]

choices = [0.13, 0.13, 0.45, 0.45, 0.45, 0.45, 0.00]

lclu_pervious['pler'] = np.select(pervious_pler_rule, choices, default=np.nan).astype(np.float32)

#now separate into what HAS a pler or what now needs to be calculated based on soils
lclu_pervious_withpler = lclu_pervious.loc[lclu_pervious['pler'] > 0]
lclu_pervious_nopler = lclu_pervious.loc[lclu_pervious['pler'] == 0]

#now break down the no_pler pervious gdf into what HAS a hydrologic soil group and what does not

#clip soils polygon to the pervious_no pler gdf
soils_clip = soils_hsg[['HYDROLGRP', 'COMPNAME', 'geometry']].clip(lclu_pervious_nopler, keep_geom_type=True)


#we can get pler from the soil layer if it has a HSG or if it's Urban or Udorthent soil
soil_types = ['Udorthents', 'Urban land']
soils_clip_withhsg_or_urbanland = soils_clip.loc[(soils_clip['HYDROLGRP'] != ' ') | ((soils_clip['HYDROLGRP'] == ' ') & (soils_clip['COMPNAME'].isin(soil_types)))]

#if the soil doesn't have a hsg and it's not urban or udorthent soil, AND it's rock outcrop, we get the pler by land use
soils_clip_rockoutcrop = soils_clip.loc[((soils_clip['HYDROLGRP'] == ' ') & (soils_clip['COMPNAME'] == 'Rock outcrop'))]


#first get pler for hsg / urban land
hsg_pler_rule = [
        (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'A') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'A/D'),
        (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'B') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'B/D'),
        (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'C') | (soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'C/D'),
        soils_clip_withhsg_or_urbanland['HYDROLGRP'] == 'D',
        soils_clip_withhsg_or_urbanland['HYDROLGRP'] == ' ' #this captures urban land or udorthent soils
]

choices = [0.03, 0.12, 0.21, 0.37, 0.21]

soils_clip_withhsg_or_urbanland['pler'] = np.select(hsg_pler_rule, choices, default=np.nan).astype(np.float32)


#to get pler for rock outcrop areas, clip lclu and run for that 
lclu_pervious_rockoutcrop = lclu_pervious_nopler.clip(soils_clip_rockoutcrop, keep_geom_type=True)

imp_pler_rule = [
    (lclu_pervious_rockoutcrop['usegencode'] == 10) | (lclu_pervious_rockoutcrop['usegencode'] == 12),
    (lclu_pervious_rockoutcrop['usegencode'] == 3) | (lclu_pervious_rockoutcrop['usegencode']==4) | (lclu_pervious_rockoutcrop['usegencode']==9) | 
        (lclu_pervious_rockoutcrop['usegencode']==20) | (lclu_pervious_rockoutcrop['usegencode']==30),
    lclu_pervious_rockoutcrop['usegencode']==55,
    (lclu_pervious_rockoutcrop['usegencode'] == 6) | (lclu_pervious_rockoutcrop['usegencode']==7) | (lclu_pervious_rockoutcrop['usegencode']==0) | 
        (lclu_pervious_rockoutcrop['usegencode']==2) | (lclu_pervious_rockoutcrop['usegencode']==8),
    (lclu_pervious_rockoutcrop['usegencode'] == 13) | (lclu_pervious_rockoutcrop['usegencode']==11)
    ]

choices = [2.32, 1.78, 1.95, 1.52, 1.96]

lclu_pervious_rockoutcrop['pler'] = np.select(imp_pler_rule, choices, default=np.nan).astype(np.float32)

#join and dissolve
joined = pd.concat([lclu_pervious_rockoutcrop[['pler', 'geometry']], 
                    soils_clip_withhsg_or_urbanland[['pler', 'geometry']], 
                    lclu_pervious_withpler[['pler', 'geometry']], 
                    lclu_impervious[['pler', 'geometry']]])


folder_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Intermediate"
path = os.path.join(folder_path) #make a subdirectory in intermediate folder w town name
os.makedirs(path, exist_ok=True)
joined.to_file(path + '\\pler_calculation.shp')



Reading in Land Cover/Land Use Data...
